## Random Forest
- 2001, Leo Breiman이 Decision Tree의 단점을 개선하기 위해 만든 알고리즘 중 하나. Decision Tree의 확장.
- Random Forest는 훌륭한 데이터 분석 알고리즘으로 꼽히며, classification, clustering, feature의 중요성 확인, prediction에 쓰임.
- Dataset에서 샘플 데이터를 선택(bagging/bootstrap aggregation)하고, 샘플 데이터를 이용해 Decision Tree를 생성, 이 과정을 n번 반복해 가장 많이 등장하는 결과를 선택.
- over-fitting 문제가 적고 구현이 간단. 병렬 계산이 간편함. 두 가지 random.
 - Random 1: Dataset에서 샘플 데이터를 random으로 선택
 - Random 2: 샘플 데이터에서 feature를 random으로 선택 -> decision tree 생성
- selected data 중복 가능

### 자기 성능 평가

- Bagging: 63% 데이터 이용해 매개 tree 생성. 나머지 37%로 tree 성능 평가.
 - 매개 tree에 입력하는 데이터는 다름

- Out-of-Bag(OOB)
 - OOB 데이터를 이용해 tree의 성능을 교정
 - OBB는 성능 통계에서 많이 사용됨


In [1]:
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
# sklearn.metrics: 모듈 성능 평가

import numpy as np
import pandas as pd

In [2]:
iris = load_iris()
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [3]:
iris.data

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [4]:
iris.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [5]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [6]:
X_train = iris.data[:-30]
y_train = iris.target[:-30]

X_test = iris.data[-30:]
y_test = iris.target[-30:]

In [7]:
y_train

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [8]:
y_test

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2])

In [9]:
# test data의 target이 전부 2(virginica)이므로 분리가 합리적이지 않다.

In [10]:
# Random Forest 분류기 생성을 위해 RandomForestClassifier 클래스를 import
# sklearn.ensemble 모듈은 분류, 회귀 및 이상 탐지를 위한 ensemble-based 방법 포함
from sklearn.ensemble import RandomForestClassifier

In [11]:
rfc = RandomForestClassifier(n_estimators=10)  # 10개의 tree를 가진 RF 생성
rfc

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

- n_estimators : Decision Tree의 개수 (default=10)
- max_features : 최대 고려하는
- feature의 개수(default=auto)
- oob_score : out-of-bag(OOB) 사용 여부(default=False)

In [12]:
rfc.fit(X_train, y_train)  # 훈련 데이터 입력해 RF 분류기 학습시킴
prediction = rfc.predict(X_test)  # 분류기에 테스트 데이터 입력해 예측(분류)
print(prediction == y_test)  #예측(분류)한 목표 값과 실제 목표 값을 비교하고 출력

[ True False  True False  True  True False False  True False  True  True
  True False False  True  True  True False  True  True  True  True  True
  True  True  True  True  True  True]


In [13]:
rfc.score(X_test, y_test)  # 성능 평가 1

0.7333333333333333

In [14]:
# 성능 평가 2
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

print("Accuracy:", accuracy_score(y_test, prediction))
print("="*60)
print(classification_report(y_test, prediction))

# Training / Test data를 잘 분리하지 못해서 분류 성능이 낮다.

Accuracy: 0.7333333333333333
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
           2       1.00      0.73      0.85        30

    accuracy                           0.73        30
   macro avg       0.50      0.37      0.42        30
weighted avg       1.00      0.73      0.85        30



C:\ProgramData\Anaconda3\envs\python_edu\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [15]:
# Training, Test data 재생성

from sklearn.model_selection import train_test_split
X = iris.data
y = iris.target
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, test_size=0.2)
print(y_test2)

[1 0 1 1 0 0 1 0 1 1 2 1 0 1 2 0 0 0 0 1 1 0 2 2 0 2 0 1 1 1]


In [16]:
rfc2 = RandomForestClassifier(n_estimators=10)
rfc2.fit(X_train2, y_train2)
prediction2 = rfc2.predict(X_test2)
print(prediction2 == y_test2)
print("Accuracy:", accuracy_score(y_test2, prediction2))
print("="*60)
print(classification_report(y_test2, prediction2))

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True False  True  True  True
  True  True  True  True  True  True]
Accuracy: 0.9666666666666667
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       1.00      0.92      0.96        13
           2       0.83      1.00      0.91         5

    accuracy                           0.97        30
   macro avg       0.94      0.97      0.96        30
weighted avg       0.97      0.97      0.97        30



In [17]:
# 분류기 성능 accuracy, precision, recall 모두 제고
# training data 잘 선택하는 것 매우 중요

In [18]:
from sklearn.model_selection import train_test_split
X = iris.data
y = iris.target
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, test_size=0.2)
print(y_test2)

rfc2 = RandomForestClassifier(n_estimators=200,
                              max_features=4,
                              oob_score=True)
rfc2.fit(X_train2, y_train2)
prediction2 = rfc2.predict(X_test2)
print(prediction2 == y_test2)
print("Accuracy:", accuracy_score(y_test2, prediction2))
print("="*60)
print(classification_report(y_test2, prediction2))

[0 1 2 2 0 1 0 0 2 0 1 1 2 1 1 2 0 1 0 0 1 1 2 2 2 0 0 1 2 2]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True False  True  True  True  True  True  True  True False  True  True
  True  True  True  True  True  True]
Accuracy: 0.9333333333333333
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.80      0.89        10
           2       0.83      1.00      0.91        10

    accuracy                           0.93        30
   macro avg       0.94      0.93      0.93        30
weighted avg       0.94      0.93      0.93        30



In [19]:
# 각 feature의 중요도 확인

for feature, imp in zip(iris.feature_names, rfc2.feature_importances_):
    print(feature, imp)
    
# 분류기 생성 시 RandomForestClassifier()의 파라미터 oob_score=True 선택.

sepal length (cm) 0.004522867704689863
sepal width (cm) 0.010505552181542954
petal length (cm) 0.48076344238112473
petal width (cm) 0.5042081377326424
